# Описание датасета, EDA

Датасет содержит информацию о клиентах банка, которые либо ушли, либо остались клиентами банка

Список фичей:

    •   Customer ID: Уникальный идентификатор каждого клиента.
    •   Surname: Фамилия клиента.
    •   Credit Score: Числовое значение, представляющее кредитный рейтинг клиента.
    •   Geography: Страна проживания клиента (Франция, Испания или Германия).
    •   Gender: Пол клиента (Мужской или Женский).
    •   Age: Возраст клиента.
    •   Tenure: Количество лет, которое клиент обслуживается в банке.
    •   Balance: Баланс на счёте клиента.
    •   NumOfProducts: Количество банковских продуктов, которыми пользуется клиент (например, сберегательный счёт, кредитная карта).
    •   HasCrCard: Наличие кредитной карты у клиента (1 = да, 0 = нет).
    •   IsActiveMember: Является ли клиент активным членом банка (1 = да, 0 = нет).
    •   EstimatedSalary: Предполагаемая заработная плата клиента.
    •   Exited: Ушёл ли клиент (1 = да, 0 = нет), таргет.

Задача - бинарная классификация, предсказывание вероятности ухода клиента для тестового набора данных

В качестве baseline построен sample_submission.csv, в котором вероятность ухода каждого клиента - 0.5


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

In [ ]:
!pip install ydata-profiling --quiet

In [ ]:
!gdown 1XCFRgfoG0mK08v1tDCgJXBcPK6PbaVQ7 # test
!gdown 1ItKuMwuaqWcHCCmJ7Wj8neNgs1PPrpLy # train
!gdown 1lX2th7npV67Qzd1NLgv3rvEJyHuu4ZBD # test_submission

### Предобработка датасета

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
target = "Exited"

In [ ]:
def encode_categorical_features(train, test=None, categorical_features=['Geography', 'Gender']):
    """
    Кодирует категориальные признаки с помощью LabelEncoder
    """
    label_encoders = {}

    for feature in categorical_features:
        le = LabelEncoder()
        # Обработка train
        train[feature] = le.fit_transform(train[feature])

        # Обработка test
        if test is not None and feature in test.columns:
            test[feature] = le.transform(test[feature])

        label_encoders[feature] = le

    if test is not None:
        return train, test, label_encoders
    else:
        return train, label_encoders

In [ ]:
train, _ = encode_categorical_features(train)

In [ ]:
to_drop = ['id', 'CustomerId', 'Surname']
train = train.drop(columns=to_drop)

## EDA

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(train, title="Final_Project")

profile.to_notebook_iframe()
profile.to_file("eda_report.html")

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
num_cols = train.select_dtypes(include=['number']).columns.tolist()

In [ ]:
# График дисбаланса классов целевой переменной
target_cnt = train[target].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=target_cnt.index, y=target_cnt.values)
plt.title('Дисбаланс классов целевой переменной')
plt.xlabel('Exited')
plt.ylabel('Количество клиентов')
plt.xticks([0, 1], ['Остался', 'Ушел'])

for i, v in enumerate(target_cnt.values):
    plt.text(i, v + 50, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Распределение фичей
num_cols_no_target = [col for col in num_cols if col != 'Exited']

train[num_cols_no_target].hist(bins=25, figsize=(15,10))
plt.tight_layout()
plt.show()

In [ ]:
nan_counts = train.isna().sum()
nan_counts

In [ ]:
# Расчет корреляции с целевой переменной
target = 'Exited'
corr = train.corr(method='pearson')
corr_target = (
    corr[target]
    .drop(target)
    .sort_values(ascending=False)
)

plt.figure(figsize=(6, 8))
sns.barplot(
    x=corr_target.values,
    y=corr_target.index,
    orient="h"
)
plt.axvline(0, color="k", linewidth=0.8)
plt.title("Target correlation")
plt.tight_layout()
plt.show()

# Работа с аномалиями и генерация признаков

# Интерпретация и диагностика моделей